# Import Frequency Domain FLIM data to OMERO
*Matt Renshaw* 
*CALM STP*
*The Francis Crick Institute*
*2025-09-18*  

**Background**: Using the Micro-Manager device adapter for the Toggel Frequency Domain FLIM (FD_FLIM) camera (Lambert Instruments) outputs the raw phase image series in the Lambert Instruments Lifa format (.fli files) as well as Micro-Manager .ome.tif files that contain acquisition metadata and fitted lifetime values as image channels. Although the fitted lifetime values can be used for basic analysis in simple FLIM experiments, they assume a single component population for the fitting that can make FRET experiments difficult to interpret. Phasor plots allow for fitting-independent analysis of FLIM data, but require processing of the phase image series. 

**Aim**: Facilitate analysis of FD-FLIM data by uploading phase image series to OMERO, linking the image data to acquisition and experimental metadata, and processing the phase data to generate s and g values for phasor plots as well as fitted lifetime values.


## Table of Contents
1. [Setup](#setup)
2. [Upload FD FLIM data to OMERO](#fli-to-omero)
3. [Read metadata and process phase series](#read-metadata-and-process-images)

## Setup <a id="setup"></a>
Import libraries

In [ ]:
import Toggel_utils as toggel
import omero_utils as omero

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import tifffile
import json
import os
from joblib import Parallel, delayed
from scipy.optimize import curve_fit

## Upload FD FLIM data to OMERO <a id="fli-to-omero"></a>
1. Define path to directory containing raw .fli files
2. Connect to OMERO
3. FLI reader to read image planes, camera dark image and metadata
4. Subtracts camera dark image from each phase image
5. Uploads phase image series to OMERO


In [ ]:
# define path to folder containing raw .fli files
# dir_path = r'C:\Users\rensham\OneDrive - The Francis Crick Institute\repositories\temp_files'
dir_path = r'N:\outputs\treismanr\fromMatt_Toggel\20250905\raw'
omero_host = "omero-training.thecrick.org" # optional: defaults to 

In [ ]:
# connect to OMERO
conn = omero.connect("rensham", host = omero_host)

In [ ]:
# process all .fli files in dir_path

folder = Path(dir_path) # string to Path

project_name = "Toggel" # store all Toggel phase series within same 'Toggel' project
dataset_name = folder.stem

project_object = omero.create_project(conn, project_name) # get or create project called project_name
project_id = project_object.getId()

dataset_object = omero.create_dataset(conn, dataset_name, project_id) # get or create dataset called dataset_name within project_id
dataset_id = dataset_object.getId()

for file in folder.glob("*.fli"):
    filename = file.stem
    acquisition_datetime = filename[0:19]
    print(filename)

    # read .fli file and get metadata    
    md, phase_images, dark_image = toggel.read_fli_file(file)
    sizeX, sizeY, n_phases, mod_frequency = (md['LAYOUT'].get(key) for key in ('x', 'y', 'phases', 'modulationFrequency'))
    sensor_gain, ref = (md['ACQUISITION SETTINGS'].get(key) for key in ('sensorGain', 'RefLifetime'))
    
    if float(ref) > 0:
        acquisition_type = "reference"
    else:
        acquisition_type = "sample"

    key_value_pairs = []
    key_value_pairs.append(["Imaging Modality", "Frequency Domain FLIM"])
    key_value_pairs.append(["acquisition_datetime", str(acquisition_datetime)])
    key_value_pairs.append(["n_phases", str(n_phases)])
    key_value_pairs.append(["modulation_frequency_Hz", str(mod_frequency)])
    key_value_pairs.append(["sensor_gain", str(sensor_gain)])
    key_value_pairs.append(["reference_lifetime_ns", str(ref)])
    key_value_pairs.append(["acquisition_type", str(acquisition_type)])

    # subtract camera dark image from phase images
    phase_series = np.empty_like(phase_images, dtype=np.float32)
    for ph, plane in enumerate(phase_images):
        phase_series[ph] = plane.astype(np.float32) - dark_image.astype(np.float32)

    # upload images to OMERO
    image_obj = omero.create_image(conn, filename, dataset_id, key_value_pairs, phase_series, channel_names=None,
                             sizeT=int(n_phases), 
                             description="FD FLIM phase images")

if (conn):
    conn.close()
    
print(f'OMERO project id for phase series images: {project_id}')    

## Read metadata and process phase series <a id="read-metadata-and-process-images"></a>
1. Select .csv file that contains paths to .ome.tif files and experimental metadata
2. Connect to OMERO
3. Define toggel_project_id: OMERO project ID that contains the phase image series datasets
4. Loops through .ome.tif files: reads metadata, identifies reference and sample phase images series for each FLIM record
5. Processes phase image series to fitted lifetime values, mean intensity, s and g values
6. Uploads processed images to OMERO, with links to sample and reference image IDs

In [ ]:
# Open file dialog and select experimental metadata .csv file
path_to_exp_info = toggel.get_filepath()

print("path to experimental metadata file:", path_to_exp_info)
# read .csv file as pandas dataframe
df = pd.read_csv(path_to_exp_info)

In [ ]:
# connect to OMERO
conn = omero.connect("rensham", host = omero_host)

In [ ]:
toggel_project_id = 102 # OMERO project ID containing the phase image series

# initialise variables
pos_dict = {}
current_ref_id = None

# get lists of phase series image names
ref_names = omero.find_image_names_in_project_kv_dict(conn, toggel_project_id, {"acquisition_type": "reference"})
sample_names = omero.find_image_names_in_project_kv_dict(conn, toggel_project_id, {"acquisition_type": "sample"})

In [ ]:
# select Frequency Domain FLIM rows
selected = df[df["Imaging Method"]=="Frequency Domain FLIM"] # select FLIM datasets
list_of_projects = selected["Project"].unique() # select unique list of "Projects". Note: the term Projects is used to align with the OMERO data structure

for project in list_of_projects:
    subset = selected[selected["Project"] == project]
    
    # generate unique full paths to MicroManager datasets
    list_of_paths = []
    unique_paths = subset[["Parent Directory", "Project", "Dataset"]].drop_duplicates().reset_index()
    for idx, row in unique_paths.iterrows():
        path = Path(os.path.join(
            str(row["Parent Directory"]),
            str(row["Project"]),
            str(row["Dataset"]))
        )
        if path.exists() & path.is_dir():
            list_of_paths.append(path)

    # create OMERO Project
    project_name = str(project)
    project_object = omero.create_project(conn, project_name) # get or create project called project_name
    project_id = project_object.getId()

    for folder in list_of_paths:
        dataset_name = folder.stem
        dataset_object = omero.create_dataset(conn, dataset_name, project_id) # get or create dataset called dataset_name within project_id
        dataset_id = dataset_object.getId()

        files = sorted(folder.glob("*.ome.tif"))

        # process individual .ome.tif files
        for file in files:
            filename = file.stem
            # load file
            tf = tifffile.TiffFile(file)
            # get summary metadata
            summary_metadata = tf.micromanager_metadata['Summary']
            nP, nT, nC, nZ, prefix, height, width, start_time = list(
                summary_metadata[key] for key in (
                    'Positions', 
                    'Frames', 
                    'Channels', 
                    'Slices', 
                    'Prefix',
                    'Height',
                    'Width',
                    'StartTime'
                    )
                )
            # get index map for page indices
            index_map = tf.micromanager_metadata['IndexMap']
        
            for t in range(nT):
                for z in range(nZ):
                    page, pos, uid = toggel.find_page_id(index_map, c=0, z=z, t=t)
                    micromanager_metadata = tf.pages[page].tags.get('MicroManagerMetadata').value
                    metadata_values = list(
                        micromanager_metadata[key] for key in (
                            'PositionName',
                            'PixelSizeUm', 
                            'ElapsedTime-ms', 
                            'Exposure-ms',
                            'ReceivedTime',
                            'Toggel-File basename', 
                            'Toggel-Frequency (MHz)', 
                            'Toggel-Number of Phases', 
                            'Toggel-Reference lifetime (ns)',
                            'Toggel-Lower intensity threshold (DN)',
                            'Nosepiece-Label',
                            'IntermediateMagnification-Magnification',
                            'XPositionUm',
                            'YPositionUm',
                            'ZPositionUm',
                            'EM_Filter-Label',
                            'EX_Filter-Label',
                            'FilterTurret1-Label'
                            )
                        )
                    # assign variables
                    pos_label, pixel_size_um, elapsed_time_ms, exposure_ms, received_time, filebasename, frequency_mhz, nPhases, ref_lifetime, int_thr, obj, interMag, pos_x, pos_y, pos_z, emFilter, exFilter, dichroicMirror = metadata_values
        
                    # read experimental metadata from .csv file
                    # get dataset-level experiment metadata
                    dat_exp_metadata = subset[subset["Dataset"]==prefix]

                    """ 
                    check for position-level experiment metadata info
                    """
                    if dat_exp_metadata.shape[0] == 0:
                        print("Can't read experimental metadata for this dataset")
                        pos_dict = {k: "empty" for k, v in pos_dict.items()}
                    else :
                        if dat_exp_metadata.shape[0] > 1:
                            pos_subset = dat_exp_metadata[dat_exp_metadata["Label"] == pos_label] 
                        else :
                            pos_subset = dat_exp_metadata
                        #pos_subset = df[(df["Dataset"]==prefix) & (df["Label"]==pos_label)] 
                        
                        # "Dataset" and "Label" values in .csv file must be the same as the prefix and pos_label values reads from the MicroManager metadata.
                        # check for single row entry
                        if pos_subset.shape[0] == 1:
                            pos_dict = pos_subset.iloc[0].to_dict()
                        else :                
                            #pos_subset = dat_exp_metadata.iloc[0]
                            pos_dict = dat_exp_metadata.iloc[0].to_dict()
                            print(f"check experimental metadata entry for {prefix} : {pos_label}.")
                            #break
                    
                    # initialise key-value pairs for adding as Map Annotation to OMERO image
                    key_value_pairs = [[str(key), str(value)] for key, value in pos_dict.items() if not (isinstance(value, float) and np.isnan(value))]
                    key_value_pairs.append(["MicroManager_filename", str(filename)])
                    key_value_pairs.append(["elapsed_time_ms", str(elapsed_time_ms)])
                    key_value_pairs.append(["exposure_ms", str(exposure_ms)])
                    key_value_pairs.append(["pixel_size_um", str(pixel_size_um)])
                    key_value_pairs.append(["pos_label", str(pos_label)])
                    key_value_pairs.append(["received_time", str(received_time)])
                    key_value_pairs.append(["position_X", str(pos_x)])
                    key_value_pairs.append(["position_Y", str(pos_y)])
                    key_value_pairs.append(["position_Z", str(pos_z)])
                    key_value_pairs.append(["T", str(t)])
                    key_value_pairs.append(["size_T", str(nT)])
                    key_value_pairs.append(["Z", str(z)])
                    key_value_pairs.append(["size_Z", str(nZ)])
                    key_value_pairs.append(["P", str(pos)])
                    key_value_pairs.append(["Toggel-Number of Phases", str(nPhases)])
                    key_value_pairs.append(["Objective", str(obj)])
                    key_value_pairs.append(["FLIM_channel_emission_filter", str(emFilter)])
                    key_value_pairs.append(["FLIM_channel_excitation_filter", str(exFilter)])
                    key_value_pairs.append(["FLIM_channel_dichroic_mirror", str(dichroicMirror)])
        
                    # find reference dataset (omero, HQL search for "acquisition_type" == 'reference', reference image before received_time
                    # find reference file based on timestamp of .ome.tif file
                    ref_name = toggel.find_phase_series_by_timestamp(ref_names, received_time, 'reference')
                    ref_id = omero.get_image_id_from_image_name (conn, ref_name, project_id=toggel_project_id)
                    reference_id = ref_id[0]

                    key_value_pairs.append(["reference_image_name", str(ref_name)])
                    key_value_pairs.append(["reference_image_id", str(reference_id)])

                    # Get system calibration values
                    if (reference_id != current_ref_id): 
                        # check to see if reference has already been used for calibration
                        # load reference image object from omero
                        ref_img_obj = conn.getObject("Image", reference_id)

                        # read reference metadata
                        ref_metadata = omero.get_key_value_metadata (ref_img_obj)
                        frequency = ref_metadata["modulation_frequency_Hz"]
                        ref_lifetime = ref_metadata["reference_lifetime_ns"]
                        n_phases = ref_metadata["n_phases"]        

                        # calculate expected values for reference lifetime with modulation frequency and number of phases
                        omega, phases, phase_delay_radians, modulation_depth = toggel.calculate_expected_values (frequency, ref_lifetime, n_phases)
                        key_value_pairs.append(["omega", str(omega)])
                        key_value_pairs.append(["frequency_Hz", str(frequency)])

                        # get phase series
                        pixels = ref_img_obj.getPrimaryPixels()
                        zct_list = [[0, 0, phase] for phase in range(int(n_phases))] # define zct_list
                        ref_phase_series = omero.get_planes(zct_list, pixels)
                        
                        # calibrate reference
                        ref_mean, A_fitted, phi_fitted, offset_fitted, phases, calibration_m, calibration_phi = toggel.calibrate_reference (ref_phase_series, phases, phase_delay_radians, modulation_depth, width, height)
                        current_ref_id = reference_id

                    

                    # find phase series (omero, HQL search for "acquisition_type" == 'sample', sample image before received_time)
                    phase_series_name = toggel.find_phase_series_by_timestamp(sample_names, received_time, 'sample')
                    phase_series_id = omero.get_image_id_from_image_name (conn, phase_series_name, project_id=toggel_project_id)
                    sample_object_id = phase_series_id[0]

                    key_value_pairs.append(["phase_series_image_name", str(phase_series_name)])
                    key_value_pairs.append(["phase_series_image_id", str(sample_object_id)])

                    image_title = f"{pos_label}_T{str(t).zfill(3)}_Z{str(z).zfill(3)}_{prefix}--ID{sample_object_id}"

                    # load phase series from omero
                    sample_img_obj = conn.getObject("Image", sample_object_id)
                    # get phase series
                    pixels = sample_img_obj.getPrimaryPixels()
                    sample_phase_series = omero.get_planes(zct_list, pixels)

                    ## DEFINE FITTING FUNCTIONS

                    # Define a sine wave function to fit the data
                    def sine_wave(x, amplitude, phase_shift, offset):
                        phase_shift = (phase_shift + 180) % 360
                        return amplitude * np.sin(x + phase_shift) + offset
                    
                    # Per-pixel fit function
                    def fit_pixel(x, y):
                        try:
                            signal = sample_phase_series[:, x, y]
                            signal_mean = signal.mean()
                    
                            # normalise signal to reference
                            normalised_signal = signal / ref_mean[x, y]
                            popt, _ = curve_fit(
                                sine_wave,
                                phases,
                                normalised_signal,
                                p0=[
                                    A_fitted[x, y],
                                    phi_fitted[x, y],
                                    offset_fitted[x, y]
                                ],
                                maxfev=1000
                            )
                            # Extract fitted parameters (Amplitude, Phase shift, average/offset)
                            sample_A_fitted, sample_phi_fitted, sample_offset_fitted = popt
                    
                            # Calibrate modulation
                            sample_mod = calibration_m[x, y] * sample_A_fitted / sample_offset_fitted
                        
                            # Calculate phase
                            sample_phi = abs(calibration_phi[x, y] + sample_phi_fitted)
                            
                            # Calculate lifetimes
                            phase_lifetime_ps = (np.tan(sample_phi) / (2 * np.pi * frequency)) * 1e12
                            mod_lifetime_ps = (np.sqrt(1 / sample_mod**2 - 1) / (2 * np.pi * frequency)) * 1e12
                    
                            # Calculate g and s values for phasor plotting    
                            g = sample_mod * np.cos(sample_phi) # g value
                            s = sample_mod * np.sin(sample_phi) # s value
                    
                            return x, y, g, s, phase_lifetime_ps, mod_lifetime_ps, signal_mean
                        except Exception:
                            return x, y, np.nan, np.nan, np.nan, np.nan, np.nan 

                    # check if processed image already exists in dataset

                    dataset_object = conn.getObject("Dataset", dataset_id)
                    image_exists = None
                    
                    if dataset_object.countChildren() > 0:
                        for child in dataset_object.listChildren():
                            if child.getName() == str(image_title):
                                image_exists = child
                                break    
                    
                    if image_exists:
                        print(f"Image: {image_title} already exists in dataset.")
                    else :
    
                        # Mask of valid pixels
                        int_min = np.min(sample_phase_series, axis=0)
                        int_max = np.max(sample_phase_series, axis=0)
    
                        valid_pixels = np.argwhere(((int_min >= 0) & (int_max <= 15000))) # exclude pixels where the lowest value in the phase images is negative and max value is close to saturation/saturated
                        
                        # Run in parallel
                        results = Parallel(n_jobs=-1, backend="loky", verbose=5)(
                            delayed(fit_pixel)(x, y) for x, y in valid_pixels
                        )
    
                        # output arrays
                        g_values_array = np.full((height, width), np.nan)
                        s_values_array = np.full((height, width), np.nan)
                        phase_tau_array = np.full((height, width), np.nan)
                        mod_tau_array = np.full((height, width), np.nan)
                        signal_int_array = np.full((height, width), np.nan)
                        
                        # populate with results
                        for x, y, g, s, phase_tau, mod_tau, sig_int in results:
                            # lifetime quality filter
                            if ((0 <= g <= 1) & (0 <= s <= 1) & (0 < phase_tau <= 24000) & (0 < mod_tau <= 24000)):
                                g_values_array[x, y] = g
                                s_values_array[x, y] = s
                                phase_tau_array[x, y] = phase_tau
                                mod_tau_array[x, y] = mod_tau
                                signal_int_array[x, y] = sig_int
    
                        # processed images to OMERO
                        
                        # ensure consistency of pixel type
                        image_list = [arr.astype(np.float32) for arr in [signal_int_array, mod_tau_array, phase_tau_array, g_values_array, s_values_array]]
                            
                        # Stack into a 3D array (C, Y, X)
                        image_stack = np.stack(image_list, axis=0)
    
                        # list of channel names
                        channel_names = ["intensity", "lifetime_modulation", "lifetime_phase", "g_values", "s_values"]
                        channel_labels = dict(enumerate(channel_names, start=1))
                                
                        # image description
                        desc = f"Frequency domain FLIM data processed from phase series from omero ID: {sample_object_id}"

                        # upload image to OMERO
                        image_obj = omero.create_image(
                            conn, image_title, dataset_id, key_value_pairs, image_stack, channel_names=channel_names, 
                            description=desc, sizeZ=1, sizeC=len(image_stack), sizeT=1, pixel_size_um = pixel_size_um
                        )

if (conn):
    conn.close()
    
print("All FLIM images processed and uploaded to OMERO.")